In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 21.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna as opt
import shap

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df = pd.read_csv("/content/drive/MyDrive/projectDatabase/ClassificacaoPorRodadas.csv", sep=';')
df.head()

,ChaveAnoTime,Ano,Rodada,Posicao,Clube,Jogos,SG,Pts,posicao final,Pontuacao Final
0,2008-Palmeiras,2008,27,1,Palmeiras,27,12,50,4,65
1,2008-Palmeiras,2008,28,1,Palmeiras,28,14,53,4,65
2,2009-Palmeiras,2009,15,1,Palmeiras,15,13,31,5,62
3,2009-Palmeiras,2009,16,1,Palmeiras,16,14,34,5,62
4,2009-Palmeiras,2009,17,1,Palmeiras,17,14,35,5,62


In [5]:
df = df[['ChaveAnoTime','Ano', 'Clube', 'Rodada', 'Posicao', 'Jogos', 'SG', 'Pts', 'posicao final', 'Pontuacao Final' ]]
df.head()

,ChaveAnoTime,Ano,Clube,Rodada,Posicao,Jogos,SG,Pts,posicao final,Pontuacao Final
0,2008-Palmeiras,2008,Palmeiras,27,1,27,12,50,4,65
1,2008-Palmeiras,2008,Palmeiras,28,1,28,14,53,4,65
2,2009-Palmeiras,2009,Palmeiras,15,1,15,13,31,5,62
3,2009-Palmeiras,2009,Palmeiras,16,1,16,14,34,5,62
4,2009-Palmeiras,2009,Palmeiras,17,1,17,14,35,5,62


In [6]:
round = 25

I am going to predict the chances of relegation of a team in the X round (just change the value of the variable round).
But there is a variable not in the df right now. I need to give to the model a variable that represents the *momentum* of the team. I want to give the points in the last 3, 5 and 10 macthes.

In [7]:
dfCurRound = df[df['Rodada']==round]
df3Round = df[df['Rodada']==(round-3)]
df5Round = df[df['Rodada']==(round-5)]
df10Round = df[df['Rodada']==(round-10)]

First, I am going to rename the columns in the current round. Then, I just need to merge by the column ChaveAnoTime.

In [8]:
dfCurRound.rename(columns={'Posicao':'CurPosicao', 'SG':'CurSG', 'Pts':'CurPts'}, inplace=True)
df3Round.rename(columns={'Posicao':'3Posicao', 'SG':'3SG', 'Pts':'3Pts'}, inplace=True)
df5Round.rename(columns={'Posicao':'5Posicao', 'SG':'5SG', 'Pts':'5Pts'}, inplace=True)
df10Round.rename(columns={'Posicao':'10Posicao', 'SG':'10SG', 'Pts':'10Pts'}, inplace=True)

<ipython-input-8-2a1d0c900f3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCurRound.rename(columns={'Posicao':'CurPosicao', 'SG':'CurSG', 'Pts':'CurPts'}, inplace=True)
<ipython-input-8-2a1d0c900f3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3Round.rename(columns={'Posicao':'3Posicao', 'SG':'3SG', 'Pts':'3Pts'}, inplace=True)
<ipython-input-8-2a1d0c900f3a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5Round.rename

In [9]:
dfCurRound = pd.merge(dfCurRound, df3Round[['ChaveAnoTime','3Pts','3SG','3Posicao']], on='ChaveAnoTime', how='left')
dfCurRound = pd.merge(dfCurRound, df5Round[['ChaveAnoTime','5Pts','5SG','5Posicao']], on='ChaveAnoTime', how='left')
dfCurRound = pd.merge(dfCurRound, df10Round[['ChaveAnoTime','10Pts','10SG','10Posicao']], on='ChaveAnoTime', how='left')
dfCurRound.head()

,ChaveAnoTime,Ano,Clube,Rodada,CurPosicao,Jogos,CurSG,CurPts,posicao final,Pontuacao Final,3Pts,3SG,3Posicao,5Pts,5SG,5Posicao,10Pts,10SG,10Posicao
0,2009-Palmeiras,2009,Palmeiras,25,1,25,15,47,5,62,41,14,1,37,13,2,31,13,1
1,2016-Palmeiras,2016,Palmeiras,25,1,25,19,48,1,80,43,18,1,39,16,1,32,17,1
2,2022-Palmeiras,2022,Palmeiras,25,1,25,23,51,1,81,48,23,1,42,19,1,29,15,1
3,2015-Corinthians,2015,Corinthians,25,1,25,23,54,1,81,49,20,1,43,16,1,30,8,2
4,2017-Corinthians,2017,Corinthians,25,1,25,21,54,1,72,50,22,1,50,24,1,37,18,1


Now the dataset is full, but I want to give the model the variation since those rounds and I am going to drop the columns I will not use in the training.

In [10]:
dfCurRound['Var3Pts'] = dfCurRound['CurPts'] - dfCurRound['3Pts']
dfCurRound['Var3SG'] = dfCurRound['CurSG'] - dfCurRound['3SG']
dfCurRound['Var3Pos'] =  dfCurRound['3Posicao'] -dfCurRound['CurPosicao']

dfCurRound['Var5Pts'] = dfCurRound['CurPts'] - dfCurRound['5Pts']
dfCurRound['Var5SG'] = dfCurRound['CurSG'] - dfCurRound['5SG']
dfCurRound['Var5Pos'] =  dfCurRound['5Posicao'] - dfCurRound['CurPosicao']

dfCurRound['Var10Pts'] = dfCurRound['CurPts'] - dfCurRound['10Pts']
dfCurRound['Var10SG'] = dfCurRound['CurSG'] - dfCurRound['10SG']
dfCurRound['Var10Pos'] =  dfCurRound['10Posicao'] - dfCurRound['CurPosicao']

dfCurRound.drop(['3Pts', '3SG', '3Posicao', '5Pts', '5SG', '5Posicao', '10Pts', '10SG', '10Posicao', 'Pontuacao Final' ], axis=1, inplace=True)
dfCurRound.head()

,ChaveAnoTime,Ano,Clube,Rodada,CurPosicao,Jogos,CurSG,CurPts,posicao final,Var3Pts,Var3SG,Var3Pos,Var5Pts,Var5SG,Var5Pos,Var10Pts,Var10SG,Var10Pos
0,2009-Palmeiras,2009,Palmeiras,25,1,25,15,47,5,6,1,0,10,2,1,16,2,0
1,2016-Palmeiras,2016,Palmeiras,25,1,25,19,48,1,5,1,0,9,3,0,16,2,0
2,2022-Palmeiras,2022,Palmeiras,25,1,25,23,51,1,3,0,0,9,4,0,22,8,0
3,2015-Corinthians,2015,Corinthians,25,1,25,23,54,1,5,3,0,11,7,0,24,15,1
4,2017-Corinthians,2017,Corinthians,25,1,25,21,54,1,4,-1,0,4,-3,0,17,3,0


Now, I am going to create the target column, if the 'posicao final' is in (17, 18, 19, 20) I will give the number 1, else 0.

In [11]:
dfCurRound['Target'] = np.where(dfCurRound['posicao final'] >= 17, 1, 0)
dfCurRound.head()

,ChaveAnoTime,Ano,Clube,Rodada,CurPosicao,Jogos,CurSG,CurPts,posicao final,Var3Pts,Var3SG,Var3Pos,Var5Pts,Var5SG,Var5Pos,Var10Pts,Var10SG,Var10Pos,Target
0,2009-Palmeiras,2009,Palmeiras,25,1,25,15,47,5,6,1,0,10,2,1,16,2,0,0
1,2016-Palmeiras,2016,Palmeiras,25,1,25,19,48,1,5,1,0,9,3,0,16,2,0,0
2,2022-Palmeiras,2022,Palmeiras,25,1,25,23,51,1,3,0,0,9,4,0,22,8,0,0
3,2015-Corinthians,2015,Corinthians,25,1,25,23,54,1,5,3,0,11,7,0,24,15,1,0
4,2017-Corinthians,2017,Corinthians,25,1,25,21,54,1,4,-1,0,4,-3,0,17,3,0,0


In [12]:
X = dfCurRound.drop(['ChaveAnoTime', 'Ano', 'Clube', 'Rodada', 'Target','posicao final'], axis=1)
y = dfCurRound['Target']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
lgb_model = lgb.LGBMClassifier(objective='regression', metric='rmse', num_leaves=31, learning_rate=0.05, n_estimators=100)

In [15]:
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 243
[LightGBM] [Info] Number of data points in the train set: 252, number of used features: 12
[LightGBM] [Info] Start training from score 0.198413
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

LGBMClassifier(learning_rate=0.05, metric='rmse', objective='regression')

In [16]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100
}

# Create LightGBM datasets
lgb_train = lgb.Dataset(X_train, y_train)

# Train the model
model = lgb.train(params, lgb_train, num_boost_round=100)

# Make predictions
y_pred = model.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 243
[LightGBM] [Info] Number of data points in the train set: 252, number of used features: 12
[LightGBM] [Info] Start training from score 0.198413


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [17]:
y_pred = model.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 0.0897247587855212
RMSE: 0.29954091337498656
MAE: 0.1538258584609503
R-squared: 0.4468553982693869


In [19]:
result = model.predict(X)

In [20]:
dfResult = pd.DataFrame(result, columns=['Prediction'])

In [21]:
dfResult['Prediction'] =  dfResult['Prediction'] * 100

In [22]:
#dfCurRound.drop('Prediction', inplace=True, axis=1)

In [23]:
dfCurRound = pd.concat([dfCurRound, dfResult], axis=1)

In [24]:
dfCurRound[ dfCurRound['Ano']==2023].sort_values(by='CurPosicao', ascending=True)

,ChaveAnoTime,Ano,Clube,Rodada,CurPosicao,Jogos,CurSG,CurPts,posicao final,Var3Pts,Var3SG,Var3Pos,Var5Pts,Var5SG,Var5Pos,Var10Pts,Var10SG,Var10Pos,Target,Prediction
17,2023-Botafogo,2023,Botafogo,25,1,25,24,52,5,1,-2,0,4,0,0,13,5,0,0,2.156728
80,2023-Bragantino,2023,Bragantino,25,2,25,13,45,6,9,5,4,13,7,5,21,7,5,0,2.528505
311,2023-Grêmio,2023,Grêmio,25,3,25,8,44,2,4,-1,0,10,4,2,17,4,0,0,-0.025752
208,2023-Palmeiras,2023,Palmeiras,25,4,25,19,44,1,3,-1,-2,7,0,-2,19,7,2,0,-1.878443
292,2023-Flamengo,2023,Flamengo,25,5,25,7,43,4,4,-2,-1,8,-1,-2,16,-2,-3,0,2.464341
280,2023-Fluminense,2023,Fluminense,25,6,25,5,41,7,3,-4,-1,7,-3,-2,16,-2,-1,0,-0.588169
265,2023-Atlético-MG,2023,Atlético-MG,25,7,25,10,40,3,9,4,2,13,6,4,19,6,5,0,-0.834169
81,2023-Athletico-PR,2023,Athletico-PR,25,8,25,8,40,8,6,2,-1,8,2,-2,17,6,0,0,1.642011
82,2023-Fortaleza,2023,Fortaleza,25,9,25,6,39,10,7,2,-1,10,3,-1,16,2,0,0,0.649208
250,2023-São Paulo,2023,São Paulo,25,10,25,4,34,11,3,-1,0,6,-1,0,9,-4,-6,0,1.060683


In [25]:
dfAtual = pd.read_csv('/content/drive/MyDrive/projectDatabase/TabelaAtual.csv', sep=';')
dfAtual.columns = ['Clube'] + dfAtual.columns[1:].tolist()
dfAtual.head()

,Clube,CurPosicao,CurSG,CurPts,Var3Pts,Var3SG,Var3Pos,Var5Pts,Var5SG,Var5Pos,Var10Pts,Var10SG,Var10Pos
0,Internacional,10,3,32,6,2,3,8,2,1,9,0,-1
1,Bragantino,11,-1,30,3,-1,0,5,-1,-1,8,-3,-1
2,Athletico-PR,12,-2,29,0,-4,-4,1,-6,-4,4,-10,-6
3,Criciúma,13,-3,28,1,-2,-1,4,-2,-1,11,-1,1
4,Juventude,14,-6,28,16,-2,0,7,-1,0,9,0,-3


In [26]:
dfJogos = pd.DataFrame([22,24,23,24,24,23,24,25,25,23,25],columns=['Jogos'])
dfAtual = pd.concat([dfAtual, dfJogos], axis=1)
dfAtual = dfAtual[['Clube','CurPosicao', 'Jogos', 'CurSG', 'CurPts', 'Var3Pts', 'Var3SG', 'Var3Pos', 'Var5Pts', 'Var5SG', 'Var5Pos', 'Var10Pts', 'Var10SG', 'Var10Pos']]

In [27]:
X_atual = dfAtual.drop('Clube', axis=1)
result = model.predict(X_atual)
dfResult = pd.DataFrame(result, columns=['Prediction (%)'])
dfResult['Prediction (%)'] =  dfResult['Prediction (%)'] * 100
dfAtual = pd.concat([dfAtual, dfResult], axis=1)
dfAtual['Prediction (%)'] = dfAtual['Prediction (%)'].apply(lambda x: 100 if x > 100 else (0 if x < 0 else x))
dfAtual['Prediction (%)'] = dfAtual['Prediction (%)'].apply(lambda x: f"{x:.2f}")
dfFinal = dfAtual[['CurPosicao','Clube', 'Jogos','CurPts','Prediction (%)']].sort_values(by='CurPosicao', ascending=True)
dfFinal.columns = ['Posicao','Clube', 'Jogos','Pts','Prediction (%)']
dfFinal

,Posicao,Clube,Jogos,Pts,Prediction (%)
0,10,Internacional,22,32,6.55
1,11,Bragantino,24,30,11.71
2,12,Athletico-PR,23,29,0.00
3,13,Criciúma,24,28,48.40
4,14,Juventude,24,28,37.28
5,15,Grêmio,23,27,25.65
6,16,Fluminense,24,27,9.22
7,17,Corinthians,25,25,11.62
8,18,Vitória,25,22,100.00
9,19,Cuiabá,23,21,86.61
